In [3]:
# import dependencies
import pandas as pd
import numpy as np

In [4]:
# load in files

lab_file = pd.read_csv("Resources_raw/lab_data_raw.csv")
rx_file = pd.read_csv("Resources_raw/reactor_data_raw.csv")
ta_file = pd.read_csv("Resources_raw/towerA_data_raw.csv")
tb_file = pd.read_csv("Resources_raw/towerB_data_raw.csv")
tc_file = pd.read_csv("Resources_raw/towerC_data_raw.csv")

# csv to dataframe

lab_data = pd.DataFrame(lab_file)
rx_data = pd.DataFrame(rx_file)
ta_data = pd.DataFrame(ta_file)
tb_data = pd.DataFrame(tb_file)
tc_data = pd.DataFrame(tc_file)

In [5]:
print(lab_data.columns)

Index(['Date', 'TA.OVH.RECYCLE.NC4.VOLPCT'], dtype='object')


In [6]:
print(rx_data.columns)

Index(['Date', 'RX.TB.RECYCLE.CHARGE.BPD', 'RX.TB.RECYCLE.CHARGE.PCT',
       'RX.NORTH.TC.CHARGE.BPD', 'RX.NORTH.TC.CHARGE.CV.PCT',
       'RX.NORTH.FLUSH.PSIG', 'RX.NORTH.OUT.F',
       'RX.SOUTH.TA.RECYCLE.CHARGE.BPD', 'RX.SOUTH.TA.RECYCLE.CHARGE.PCT',
       'RX.SOUTH.TC.CHARGE.BPD', 'RX.SOUTH.TC.CHARGE.CV.PCT',
       'RX.SOUTH.FLUSH.PSIG', 'RX.SOUTH.OUT.F', 'RX.SOUTH.WATERRETURN.F',
       'RX.SOUTH.WATERRETURN.PH', 'RX.OUT.DELTA.F'],
      dtype='object')


In [67]:
def clean_data(df):
    
    # make list of df columns
    df_columns = list(df)
    
    # convert necessary objects to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # add hour column for sorting invalid checkpoints
    df['Hour'] = pd.to_datetime(df['Date']).dt.hour
    
    # keep only valid hour checkpoints
    df = df.loc[(df['Hour'] == 6) | (df['Hour'] == 13)]
    
    # store start and end dates of turnaround as variables
    start = "2018-10-12 13:00:00"
    end = "2019-03-29 6:00:00"
    
    # dates between start and end converted to null
    df.loc[(df['Date'] > start) & (df['Date'] <= end)] = np.nan
    
    # for loop to nullify text strings
    for name in df_columns:
        df.loc[df[name] == "I/O Timeout"] = np.nan
        df.loc[df[name] == "Error"] = np.nan
        df.loc[df[name] == "Bad"] = np.nan
        df.loc[df[name] == "Bad Input"] = np.nan
        df.loc[df[name] == "No Data"] = np.nan
        
    # drop nulls
    df = df.dropna()
    
    # convert all other object types to float
    df = df.astype(dict.fromkeys(df.columns[1:-1], float))
    
    # reset index
    df = df.reset_index()
    
    # drop unneeded columns (index and hour)
    df = df.drop(columns=['Hour', 'index'])
    
    return df

In [68]:
# clean lab data
lab_data = clean_data(lab_data)

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [69]:
lab_data.describe()

,TA.OVH.RECYCLE.NC4.VOLPCT
count,5342.000000
mean,11.136153
std,14.474349
min,0.000000
25%,8.200000
50%,10.300000
75%,12.800000
max,1005.000000


In [70]:
# clean reactor data
rx_data = clean_data(rx_data)
rx_data.describe()

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,RX.TB.RECYCLE.CHARGE.BPD,RX.TB.RECYCLE.CHARGE.PCT,RX.NORTH.TC.CHARGE.BPD,RX.NORTH.TC.CHARGE.CV.PCT,RX.NORTH.FLUSH.PSIG,RX.NORTH.OUT.F,RX.SOUTH.TA.RECYCLE.CHARGE.BPD,RX.SOUTH.TA.RECYCLE.CHARGE.PCT,RX.SOUTH.TC.CHARGE.BPD,RX.SOUTH.TC.CHARGE.CV.PCT,RX.SOUTH.FLUSH.PSIG,RX.SOUTH.OUT.F,RX.SOUTH.WATERRETURN.F,RX.SOUTH.WATERRETURN.PH,RX.OUT.DELTA.F
count,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000
mean,2761.123859,28.004040,2685.611161,23.140521,131.203216,91.476504,6768.491359,55.275676,2068.195815,18.320759,131.427272,92.292182,78.365939,7.209025,-0.797493
std,1097.880255,10.762031,550.593059,8.099408,13.792705,7.731235,1020.313107,4.901060,437.958774,5.009712,20.062681,7.517265,7.345467,1.884970,3.335101
min,0.000000,-6.900000,0.000000,-6.900001,0.000000,39.115906,0.000000,-6.900001,0.000000,-6.900001,0.000000,53.540039,51.538086,-0.089193,-47.111736
25%,2046.409607,18.000000,2416.367248,18.177530,124.570370,86.833942,6314.628174,53.140571,1864.641632,16.280122,118.794918,87.878038,74.310684,6.520175,-2.876673
50%,2635.391357,27.163007,2678.655395,21.217908,127.410889,92.583805,6508.248535,55.221201,2104.441407,18.652915,124.353680,93.508320,80.143162,7.547871,-0.276517
75%,3465.538757,35.668804,2992.078308,29.855751,133.004734,97.124060,6843.628052,57.215943,2343.136658,21.153531,141.758537,97.777939,83.795166,8.324176,0.891529
max,7763.305664,105.000000,4402.867188,100.000000,252.685547,110.000893,16473.312500,68.686150,3701.582275,45.000000,252.983093,109.204102,93.135925,14.439209,46.707916


In [71]:
# clean Tower A data
ta_data = clean_data(ta_data)
ta_data.describe()

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TA.Charge.BPD,TA.OH.PSIG,TA.Tray4.F,TA.OH.F,TA.RECV.PCT,TA.RECV.PSIG,TA.REFLUX.BPD,TA.RECYCLE.TO.CONTACT.BPD,TA.RECYCLE.TO.CONTACT.F,TA.TRAY39.F,TA.TRAY45.F,TA.DP.PSIG,TA.BTM.PSIG,TA.BTM.LEVEL.PCT,TA.BTM.REBOIL.OUT.F,TA.TOP.REBOIL.OUT.F
count,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000,4432.000000
mean,15624.428569,97.724862,134.155713,131.522996,9.290074,100.181961,6992.693161,6726.644595,84.438646,164.226270,178.433037,7.278620,103.291009,90.847109,309.604144,210.518725
std,4264.826486,6.224325,3.587819,4.000799,9.003224,4.538600,1307.059555,989.960174,8.754896,6.900231,10.476421,9.096112,10.362998,12.979410,12.242742,17.863358
min,0.000000,-1.532560,102.691650,102.038574,-6.900000,57.708740,-836.645019,0.000000,27.513123,131.103516,136.929321,0.002177,3.174019,0.000000,185.351562,150.372314
25%,12189.267090,96.931414,132.427978,129.534149,6.732341,99.589539,6351.218872,6297.223755,79.359818,160.897350,174.343872,5.144447,99.865723,82.110928,301.676941,197.711182
50%,15698.013670,97.401123,133.938599,131.687927,9.928573,100.012207,7143.522216,6497.671875,85.122681,164.300552,177.410904,7.573607,104.711151,87.989479,310.391235,207.223518
75%,17979.037597,99.453428,135.274635,133.263462,13.606769,100.360107,7898.555787,6777.425781,89.757053,167.592548,181.070332,8.386230,106.025507,102.352913,319.514465,219.818130
max,29810.134770,133.076477,194.899170,194.660858,105.000000,136.087036,12522.957030,16473.312500,113.681038,299.072266,331.774933,101.661385,141.533661,199.996948,359.384460,329.602081


In [72]:
tb_data = clean_data(tb_data)
tb_data.describe()
# note that lb/hr means pounds per hour

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TB.CHARGE.BPD,TB.CHARGE.PREHEAT.COND.LBHR,TB.CHARGE.F,TB.OVHD.F,TB.OVHD.PSIG,TB.OVHD.REVEIVER.PSIG,TB.OVHD.REVEIVER.F,TB.OVHD.REVEIVER.LVL.PCT,TB.BTM.PSIG,TB.BTM.LVL.PCT,TB.DP.PSIG,TB.BTM.F,TB.REBOIL.OUT.F
count,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000
mean,12772.576421,3647.188501,145.618049,175.237906,183.310000,184.814844,75.782512,61.112795,186.900032,82.212531,3.593235,213.325288,216.348596
std,1064.448754,787.050647,7.450779,5.093775,2.388552,3.063238,19.327296,46.662511,4.035574,4.265610,3.131000,5.145277,4.903160
min,5858.637207,914.653503,123.025520,154.954483,157.860306,155.394135,19.893646,-91.756966,155.940903,63.886410,-2.833572,196.509537,199.824417
25%,12141.566892,3093.009827,138.095253,172.446308,182.816875,183.020649,62.677482,40.789262,184.409168,79.995989,1.729459,209.914875,213.086197
50%,12822.943845,3731.380493,145.350647,175.749786,183.068848,184.525085,77.493286,45.076784,186.549385,84.102146,3.474136,212.543175,216.151367
75%,13393.766843,4221.606689,152.993515,178.372959,183.994247,185.989403,89.450455,67.708408,188.581070,85.054365,4.949290,217.817627,220.166130
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,199.990845,218.297592,110.159851,27.846790,248.548935,257.684631


In [73]:
tc_data = clean_data(tc_data)
tc_data.describe()

C:\Users\anna\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,TC.TRAY7.F,TC.TRAY40.F,TC.CHARGE.F,TC.OVHD.PSIG,TC.OVHD.RECVR.F,TC.TRAY24.F,TC.DP.PSIG,TC.BTM.PSIG,TC.REBOIL.OUT.F,TC.REBOIL.COND.LBHR
count,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000
mean,192.221469,204.439332,166.386492,238.913951,171.965331,188.931519,3.278840,242.184423,206.660325,7025.295286
std,4.610535,4.305985,15.046051,9.563152,8.414807,13.632097,0.514274,9.620863,4.051974,1058.005927
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641,50.172615
25%,191.162556,203.845978,158.068848,236.864265,167.680370,173.988453,3.032314,240.154251,205.786133,6343.553223
50%,192.697708,204.798286,164.086914,239.973762,172.128960,195.822716,3.288918,243.133560,206.711647,7005.793213
75%,193.999901,205.913528,170.924381,242.884811,177.360535,197.891464,3.519506,246.192932,207.887211,7751.950928
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584,11856.028320


# Handling Outliers

#### Group Discussion 
- drop rows where RECV.PCT is outside 0-100, indicates invalid
- drop entire Reflux column, an untrusted measure industry wide according to John
- Recycle measures should never be negative, so drop rows where negative

In [74]:
# define find outlier function
def find_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(.75)
    iqr = q3-q1
    lower_lim = q1 - (1.5 * iqr)
    upper_lim = q3 + (1.5 * iqr)
    outliers = df[((df<lower_lim) | (df>upper_lim))]
    return outliers

### Lab Outliers

In [75]:
# outliers in lab data
lab_outliers = find_outliers(lab_data['TA.OVH.RECYCLE.NC4.VOLPCT'])
lab_outliers.describe()

count     224.000000
mean       24.769196
std        67.705369
min         0.000000
25%        19.799999
50%        21.799999
75%        26.125000
max      1005.000000
Name: TA.OVH.RECYCLE.NC4.VOLPCT, dtype: float64

In [76]:
# set upper limit of 40 for valid values in lab data
lab_data.loc[lab_data['TA.OVH.RECYCLE.NC4.VOLPCT'] >= 40] = np.nan
lab_outliers = find_outliers(lab_data['TA.OVH.RECYCLE.NC4.VOLPCT'])
lab_outliers.describe()

count    211.000000
mean      18.107109
std       11.065566
min        0.000000
25%        1.100000
50%       21.600000
75%       25.250000
max       36.799999
Name: TA.OVH.RECYCLE.NC4.VOLPCT, dtype: float64

### Reactor Outliers

In [77]:
# outliers in reactor data
rx_float = rx_data.iloc[:, 1:-1]
rx_outliers = find_outliers(rx_float)
rx_outliers.describe()

,RX.TB.RECYCLE.CHARGE.BPD,RX.TB.RECYCLE.CHARGE.PCT,RX.NORTH.TC.CHARGE.BPD,RX.NORTH.TC.CHARGE.CV.PCT,RX.NORTH.FLUSH.PSIG,RX.NORTH.OUT.F,RX.SOUTH.TA.RECYCLE.CHARGE.BPD,RX.SOUTH.TA.RECYCLE.CHARGE.PCT,RX.SOUTH.TC.CHARGE.BPD,RX.SOUTH.TC.CHARGE.CV.PCT,RX.SOUTH.FLUSH.PSIG,RX.SOUTH.OUT.F,RX.SOUTH.WATERRETURN.F,RX.SOUTH.WATERRETURN.PH
count,15.000000,5.000000,124.000000,60.000000,705.000000,41.000000,749.000000,155.000000,115.000000,136.000000,116.000000,54.000000,90.000000,381.000000
mean,6223.321908,86.409399,1855.541644,-0.172941,151.041288,65.859691,8040.687895,54.443368,813.283946,5.624394,158.409980,69.374049,57.363842,2.863460
std,616.697033,22.077870,1587.513709,20.335159,22.155998,7.258679,1806.014646,21.401906,1249.253469,16.006159,66.713228,4.555457,2.004713,1.806390
min,5624.084473,62.445980,0.000000,-6.900001,0.000000,39.115906,0.000000,-6.900001,0.000000,-6.900001,0.000000,53.540039,51.538086,-0.089193
25%,5792.404297,62.524685,518.433357,-6.900000,148.834228,64.468620,7952.968750,46.237272,5.901617,-6.900000,177.064938,67.960739,55.814365,2.353659
50%,6126.708984,97.076332,1352.204163,-6.900000,151.706696,68.870544,8236.875977,64.469925,10.188961,-3.444444,179.643044,71.173859,57.961275,2.672119
75%,6397.888184,105.000000,3933.201782,-6.500000,156.762741,70.547485,8702.639648,66.946980,1027.408783,8.833701,184.145519,72.438751,59.090427,2.993232
max,7763.305664,105.000000,4402.867188,100.000000,252.685547,71.394348,16473.312500,68.686150,3701.582275,45.000000,252.983093,73.013611,60.006714,14.439209


In [91]:
# null invalid values
rx_data.loc[(rx_data['RX.NORTH.TC.CHARGE.CV.PCT'] < 0) | (rx_data['RX.NORTH.TC.CHARGE.CV.PCT'] > 100)] = np.nan
rx_data.loc[(rx_data['RX.TB.RECYCLE.CHARGE.PCT'] < 0) | (rx_data['RX.TB.RECYCLE.CHARGE.PCT'] > 100)] = np.nan
rx_data.loc[(rx_data['RX.SOUTH.TA.RECYCLE.CHARGE.PCT'] < 0) | (rx_data['RX.SOUTH.TA.RECYCLE.CHARGE.PCT'] > 100)] = np.nan
rx_data.loc[(rx_data['RX.SOUTH.TC.CHARGE.CV.PCT'] < 0) | (rx_data['RX.SOUTH.TC.CHARGE.CV.PCT'] > 100)] = np.nan

In [92]:
# drop nulls and check dataframe
rx_data = rx_data.dropna()
rx_data.describe()

,RX.TB.RECYCLE.CHARGE.BPD,RX.TB.RECYCLE.CHARGE.PCT,RX.NORTH.TC.CHARGE.BPD,RX.NORTH.TC.CHARGE.CV.PCT,RX.NORTH.FLUSH.PSIG,RX.NORTH.OUT.F,RX.SOUTH.TA.RECYCLE.CHARGE.BPD,RX.SOUTH.TA.RECYCLE.CHARGE.PCT,RX.SOUTH.TC.CHARGE.BPD,RX.SOUTH.TC.CHARGE.CV.PCT,RX.SOUTH.FLUSH.PSIG,RX.SOUTH.OUT.F,RX.SOUTH.WATERRETURN.F,RX.SOUTH.WATERRETURN.PH,RX.OUT.DELTA.F
count,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000,3984.000000
mean,2745.169356,27.905856,2712.308796,23.520760,131.554583,91.666790,6805.353900,55.527355,2101.256111,18.739477,131.802199,92.615661,78.489365,7.185814,-0.929161
std,1083.341435,10.555629,493.351870,7.296296,11.636061,7.409332,919.352426,3.344543,339.497271,3.731582,17.842920,7.195649,7.334693,1.894548,2.294447
min,0.000000,0.000000,784.665832,0.712704,103.017799,62.329102,0.000000,26.000000,0.000000,0.000000,34.842491,57.240231,51.538086,-0.089193,-20.792578
25%,2043.205262,18.000000,2427.171936,18.294074,124.627111,87.071611,6320.064087,53.197642,1879.365021,16.417256,118.904596,88.429260,74.520874,6.512221,-2.882713
50%,2625.228760,27.054565,2684.985840,21.294422,127.416611,92.676548,6509.127930,55.262606,2112.104493,18.722295,124.403862,93.739803,80.329319,7.527377,-0.365919
75%,3442.376770,35.502649,3000.699219,29.940923,132.888695,97.159849,6846.561279,57.255402,2345.587891,21.189699,141.752052,97.884270,83.871847,8.315765,0.876095
max,7322.576172,97.076332,4402.867188,50.219547,242.542267,110.000893,16473.312500,68.686150,3565.502930,45.000000,203.495590,109.204102,93.135925,14.439209,46.707916


In [93]:
# recheck outliers
rx_float = rx_data.iloc[:, 1:-1]
rx_outliers = find_outliers(rx_float)
rx_outliers.describe()

,RX.TB.RECYCLE.CHARGE.BPD,RX.TB.RECYCLE.CHARGE.PCT,RX.NORTH.TC.CHARGE.BPD,RX.NORTH.TC.CHARGE.CV.PCT,RX.NORTH.FLUSH.PSIG,RX.NORTH.OUT.F,RX.SOUTH.TA.RECYCLE.CHARGE.BPD,RX.SOUTH.TA.RECYCLE.CHARGE.PCT,RX.SOUTH.TC.CHARGE.BPD,RX.SOUTH.TC.CHARGE.CV.PCT,RX.SOUTH.FLUSH.PSIG,RX.SOUTH.OUT.F,RX.SOUTH.WATERRETURN.F,RX.SOUTH.WATERRETURN.PH
count,10.000000,3.000000,78.000000,6.000000,686.000000,34.000000,706.000000,127.000000,34.000000,57.000000,94.000000,50.000000,94.000000,381.000000
mean,5982.541455,74.015666,2745.026429,40.912358,153.409835,69.600737,8310.526169,61.703054,1533.754409,17.086239,180.233868,71.360900,57.639298,2.863460
std,529.373379,19.971162,1340.613673,19.719459,9.161391,2.498513,1254.540299,10.086309,1079.931925,14.103956,16.114441,3.475493,2.027629,1.806390
min,5564.300781,62.445980,784.665832,0.712704,103.017799,62.329102,0.000000,26.000000,0.000000,0.000000,34.842491,57.240231,51.538086,-0.089193
25%,5664.665039,62.485332,1429.658661,47.756273,148.782696,68.767166,7986.705078,63.872515,957.723740,6.000000,178.098679,71.166611,56.107330,2.353659
50%,5792.404297,62.524685,3881.723755,48.639271,151.665794,70.469666,8262.653320,65.043396,1114.667725,8.948753,180.198029,72.653961,58.210758,2.672119
75%,6109.588623,79.800508,3990.503784,49.510879,156.538754,71.272278,8791.425293,67.100548,2595.467621,29.116064,184.056740,73.359299,59.273149,2.993232
max,7322.576172,97.076332,4402.867188,50.219547,242.542267,71.833801,16473.312500,68.686150,3565.502930,45.000000,203.495590,74.223328,60.409546,14.439209


### Tower A Outliers

In [78]:
# Tower A outliers
ta_float = ta_data.iloc[:, 1:-1]
ta_outliers = find_outliers(ta_float)
ta_outliers.describe()

,TA.Charge.BPD,TA.OH.PSIG,TA.Tray4.F,TA.OH.F,TA.RECV.PCT,TA.RECV.PSIG,TA.REFLUX.BPD,TA.RECYCLE.TO.CONTACT.BPD,TA.RECYCLE.TO.CONTACT.F,TA.TRAY39.F,TA.TRAY45.F,TA.DP.PSIG,TA.BTM.PSIG,TA.BTM.LEVEL.PCT,TA.BTM.REBOIL.OUT.F
count,41.000000,949.000000,334.000000,286.000000,665.000000,1538.000000,172.000000,814.000000,110.000000,96.000000,178.000000,631.000000,255.000000,57.000000,33.000000
mean,17700.180372,96.437474,139.539384,133.438247,-0.987673,100.473771,4308.519581,7986.333103,75.129156,167.194391,199.327128,7.829190,100.460613,72.849561,291.339584
std,13488.224614,12.995649,8.944124,11.769573,16.949805,7.677170,3383.735933,1752.429110,22.847436,33.895166,40.552331,23.522877,40.439689,61.702114,47.614630
min,0.000000,-1.532560,102.691650,102.038574,-6.900000,57.708740,-836.645019,0.000000,27.513123,131.103516,136.929321,0.002177,3.174019,0.000000,185.351562
25%,43.794273,92.136383,139.643097,122.940832,-6.900000,95.019531,2154.883484,7818.320435,60.141754,147.636414,163.650513,0.104206,115.336994,21.635437,270.324707
50%,26958.464840,92.500305,141.523743,139.385986,-6.900000,97.713470,3206.244751,8201.000977,62.731173,149.954224,192.954834,0.180817,117.974853,39.322662,272.357178
75%,27801.376950,105.022430,143.545532,141.210937,-6.900000,104.817200,3903.295838,8594.729981,105.738394,180.420689,201.997387,0.245193,122.032932,142.654419,348.138428
max,29810.134770,133.076477,194.899170,194.660858,105.000000,136.087036,12522.957030,16473.312500,113.681038,299.072266,331.774933,101.661385,141.533661,199.996948,359.384460


In [79]:
# null out dataset specific invalid values 
ta_data.loc[(ta_data['TA.RECV.PCT'] < 0) | (ta_data['TA.RECV.PCT'] > 100)] = np.nan
ta_data.loc[ta_data['TA.Charge.BPD'] < 5000] = np.nan
ta_data.loc[ta_data['TA.RECYCLE.TO.CONTACT.BPD'] < 1000] = np.nan
ta_data.loc[(ta_data['TA.BTM.LEVEL.PCT'] < 0) | (ta_data['TA.BTM.LEVEL.PCT'] > 100)] = np.nan

In [80]:
# drop nulls
ta_data = ta_data.dropna()
ta_data.describe()

,TA.Charge.BPD,TA.OH.PSIG,TA.Tray4.F,TA.OH.F,TA.RECV.PCT,TA.RECV.PSIG,TA.REFLUX.BPD,TA.RECYCLE.TO.CONTACT.BPD,TA.RECYCLE.TO.CONTACT.F,TA.TRAY39.F,TA.TRAY45.F,TA.DP.PSIG,TA.BTM.PSIG,TA.BTM.LEVEL.PCT,TA.BTM.REBOIL.OUT.F,TA.TOP.REBOIL.OUT.F
count,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000,2546.000000
mean,13581.226564,97.587107,133.368885,130.993651,12.191080,99.894853,6974.542072,6437.945549,83.763715,163.530380,178.899115,6.968114,101.580655,82.201310,316.089147,206.034508
std,3118.119523,5.398608,2.173454,2.990626,6.556598,0.977370,1305.685968,525.975330,7.362435,5.297077,6.641016,11.786154,11.974523,8.621321,8.202325,13.753964
min,5245.315430,-1.532560,124.008186,116.775520,0.057902,88.854980,-535.964356,2651.605713,54.576115,144.874649,161.386108,0.002177,3.174019,0.000000,278.997803,174.914551
25%,11102.405520,97.143854,132.234192,129.747009,8.958455,99.813562,6502.973877,6274.797119,79.589844,160.389709,175.003052,0.375391,99.817751,79.637146,310.722725,196.352386
50%,12901.359375,97.443008,133.589172,131.848144,11.660881,100.009163,7165.480225,6451.078125,84.451801,163.805000,178.778534,7.617188,103.851700,84.366089,317.794800,203.393570
75%,15862.798585,99.104103,134.766972,132.995617,15.454600,100.195312,7688.443116,6520.216431,88.112961,167.251793,182.182857,8.537292,105.797768,85.597305,321.911621,213.596851
max,29810.134770,109.439850,145.645142,140.570068,83.057693,112.509163,12522.957030,16473.312500,108.530273,177.632065,255.816666,101.661385,127.102669,99.871826,356.948029,271.478272


In [85]:
# drop reflux column
ta_data = ta_data.drop(columns=['TA.REFLUX.BPD'])

In [86]:
# recheck outliers after dropping invalids
ta_float = ta_data.iloc[:, 1:-1]
ta_outliers = find_outliers(ta_float)
ta_outliers.describe()

,TA.Charge.BPD,TA.OH.PSIG,TA.Tray4.F,TA.OH.F,TA.RECV.PCT,TA.RECV.PSIG,TA.RECYCLE.TO.CONTACT.BPD,TA.RECYCLE.TO.CONTACT.F,TA.TRAY39.F,TA.TRAY45.F,TA.DP.PSIG,TA.BTM.PSIG,TA.BTM.LEVEL.PCT,TA.BTM.REBOIL.OUT.F
count,19.000000,54.000000,86.000000,172.000000,50.000000,323.000000,157.000000,85.000000,29.000000,83.000000,52.000000,64.000000,150.000000,30.000000
mean,25611.716078,80.752185,131.368875,123.680621,34.990243,98.991712,6902.864448,84.550297,148.964808,185.795882,81.372678,44.031318,69.274116,294.764314
std,2224.087842,32.143951,6.071902,3.051906,15.063984,2.468638,1960.231966,20.392563,5.675067,19.928741,22.118230,44.348801,26.821873,15.368648
min,23086.044920,-1.532560,124.008186,116.775520,25.298895,88.854980,2651.605713,54.576115,144.874649,161.386108,20.792770,3.174019,0.000000,278.997803
25%,23969.040040,91.533857,127.500155,122.608948,26.361126,97.935734,5651.323730,64.535522,147.250366,163.491821,74.184895,3.243801,62.910081,290.180168
50%,24803.482420,92.546082,128.153992,123.696899,28.876105,98.980637,5904.003418,101.150459,148.110718,193.983215,94.001663,23.390293,69.876862,292.404175
75%,27095.091800,93.501669,138.823917,124.354553,35.196927,100.986183,7317.470703,103.171722,149.037933,198.274277,94.335812,87.094116,95.266724,293.598938
max,29810.134770,109.439850,145.645142,140.570068,83.057693,112.509163,16473.312500,108.530273,177.632065,255.816666,101.661385,127.102669,99.871826,356.948029


### Tower B Outliers

In [83]:
# Tower B outliers
tb_float = tb_data.iloc[:, 1:-1]
tb_outliers = find_outliers(tb_float)
tb_outliers.describe()

,TB.CHARGE.BPD,TB.CHARGE.PREHEAT.COND.LBHR,TB.CHARGE.F,TB.OVHD.F,TB.OVHD.PSIG,TB.OVHD.REVEIVER.PSIG,TB.OVHD.REVEIVER.F,TB.OVHD.REVEIVER.LVL.PCT,TB.BTM.PSIG,TB.BTM.LVL.PCT,TB.DP.PSIG,TB.BTM.F
count,42.000000,4.000000,1.000000,49.000000,336.000000,107.000000,16.000000,215.000000,65.000000,22.000000,45.000000,12.000000
mean,11480.498152,4977.103561,185.386932,161.901258,183.292304,190.137803,126.420147,141.342127,197.276243,70.389702,15.877803,205.069527
std,3348.236932,2739.297348,NaN,9.354830,5.048110,7.766866,28.486220,82.272784,10.757208,9.423146,5.301888,19.419842
min,5858.637207,914.653503,185.386932,154.954483,157.860306,155.394135,19.893646,-91.756966,155.940903,63.886410,10.317993,196.509537
25%,9221.930176,4681.486618,185.386932,157.891846,179.644089,190.818596,131.841183,123.170505,195.703110,64.787657,11.676937,196.602062
50%,10084.878905,6047.801025,185.386932,160.096832,180.315941,192.013550,133.478470,183.132752,198.195862,70.378967,13.531866,196.756477
75%,15484.844730,6343.417969,185.386932,162.020874,187.259766,194.374870,134.030819,199.990845,202.056900,71.240433,19.363403,197.046520
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,199.990845,218.297592,110.159851,27.846790,248.548935


In [87]:
# remove invalid rows from dataset (anything PCT needs to be between 0-100)
tb_data.loc[(tb_data['TB.OVHD.REVEIVER.LVL.PCT'] < 0) | (tb_data['TB.OVHD.REVEIVER.LVL.PCT'] > 100)] = np.nan
tb_data.loc[(tb_data['TB.BTM.LVL.PCT'] < 0) | (tb_data['TB.BTM.LVL.PCT'] > 100)] = np.nan

In [88]:
# drop nulls and check dataframe again
tb_data = tb_data.dropna()
tb_data.describe()

,TB.CHARGE.BPD,TB.CHARGE.PREHEAT.COND.LBHR,TB.CHARGE.F,TB.OVHD.F,TB.OVHD.PSIG,TB.OVHD.REVEIVER.PSIG,TB.OVHD.REVEIVER.F,TB.OVHD.REVEIVER.LVL.PCT,TB.BTM.PSIG,TB.BTM.LVL.PCT,TB.DP.PSIG,TB.BTM.F,TB.REBOIL.OUT.F
count,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000
mean,12888.041948,3606.667269,145.199813,174.998975,183.264743,184.963378,77.269241,47.744934,186.898937,82.056868,3.648800,213.244295,216.401481
std,1047.337377,773.070854,7.393804,4.726962,2.265965,2.808644,18.705931,18.759088,3.719031,4.435992,2.857346,5.187898,4.899155
min,5858.637207,914.653503,123.025520,154.954483,173.002945,170.526138,19.893646,0.000000,173.029419,63.886410,-2.833572,196.509537,199.824417
25%,12304.933353,3070.760315,138.064068,172.332779,182.793419,183.076859,65.463675,40.154741,184.677052,79.788177,2.040822,209.905628,213.185089
50%,12931.493655,3686.642212,144.931732,175.427246,183.068848,184.658653,78.659184,44.763899,186.545570,84.248760,3.517921,212.338257,216.130386
75%,13492.972655,4158.917114,152.969051,177.870483,183.984573,186.053436,89.846333,56.918225,188.554207,85.078440,4.985080,217.725941,220.143753
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,99.378967,218.297592,91.452347,27.846790,248.548935,257.684631


In [89]:
# recheck outliers
tb_float = tb_data.iloc[:, 1:-1]
tb_outliers = find_outliers(tb_float)
tb_outliers.describe()

,TB.CHARGE.BPD,TB.CHARGE.PREHEAT.COND.LBHR,TB.CHARGE.F,TB.OVHD.F,TB.OVHD.PSIG,TB.OVHD.REVEIVER.PSIG,TB.OVHD.REVEIVER.F,TB.OVHD.REVEIVER.LVL.PCT,TB.BTM.PSIG,TB.BTM.LVL.PCT,TB.DP.PSIG,TB.BTM.F
count,39.000000,4.000000,1.000000,40.000000,294.000000,80.000000,22.000000,153.000000,53.000000,18.000000,33.000000,12.000000
mean,11675.613169,4977.103561,185.386932,163.953601,183.080209,192.041041,108.048514,52.923849,197.469664,67.920769,13.479817,205.069527
std,3402.555396,2739.297348,NaN,10.545475,4.750508,5.511911,46.808270,41.191007,9.375584,3.125278,8.848158,19.419842
min,5858.637207,914.653503,185.386932,154.954483,173.002945,170.526138,19.893646,0.000000,173.029419,63.886410,-2.833572,196.509537
25%,9266.100097,4681.486618,185.386932,159.595379,179.610222,191.193809,127.306509,6.935675,194.945999,64.745235,10.469055,196.602062
50%,10180.187500,6047.801025,185.386932,161.683586,180.163246,192.365654,132.110596,82.911995,197.442642,69.563980,12.091387,196.756477
75%,15497.792970,6343.417969,185.386932,163.502655,187.070847,194.493809,133.741280,88.106064,201.594116,70.649614,19.308472,197.046520
max,16790.464840,6898.158691,185.386932,210.779907,197.756378,200.649857,137.485886,99.378967,218.297592,71.662560,27.846790,248.548935


### Tower C Outliers

In [84]:
# Tower C outliers
tc_float = tc_data.iloc[:, 1:-1]
tc_outliers = find_outliers(tc_float)
tc_outliers.describe()

,TC.TRAY7.F,TC.TRAY40.F,TC.CHARGE.F,TC.OVHD.PSIG,TC.OVHD.RECVR.F,TC.TRAY24.F,TC.DP.PSIG,TC.BTM.PSIG,TC.REBOIL.OUT.F
count,152.000000,266.000000,551.000000,266.000000,115.000000,14.000000,156.000000,261.000000,244.000000
mean,178.240995,195.843639,187.103888,218.227422,141.746433,141.016376,3.624299,221.062942,203.614664
std,19.985205,15.850930,28.401021,32.011793,17.351797,50.253124,2.180474,32.543254,17.394623
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641
25%,180.039295,198.388329,192.583275,221.793205,136.433159,122.065285,2.170433,224.972626,201.150467
50%,185.117188,199.827560,195.742798,224.469307,145.364380,132.572472,2.284555,227.857727,201.975967
75%,186.369782,200.394325,197.219856,226.440285,150.180054,134.632637,4.545918,229.560928,202.542221
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584


In [90]:
tc_data.describe()

,TC.TRAY7.F,TC.TRAY40.F,TC.CHARGE.F,TC.OVHD.PSIG,TC.OVHD.RECVR.F,TC.TRAY24.F,TC.DP.PSIG,TC.BTM.PSIG,TC.REBOIL.OUT.F,TC.REBOIL.COND.LBHR
count,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000,5260.000000
mean,192.221469,204.439332,166.386492,238.913951,171.965331,188.931519,3.278840,242.184423,206.660325,7025.295286
std,4.610535,4.305985,15.046051,9.563152,8.414807,13.632097,0.514274,9.620863,4.051974,1058.005927
min,86.463882,97.879448,37.571716,58.603485,74.485779,84.071915,0.017766,58.572769,98.681641,50.172615
25%,191.162556,203.845978,158.068848,236.864265,167.680370,173.988453,3.032314,240.154251,205.786133,6343.553223
50%,192.697708,204.798286,164.086914,239.973762,172.128960,195.822716,3.288918,243.133560,206.711647,7005.793213
75%,193.999901,205.913528,170.924381,242.884811,177.360535,197.891464,3.519506,246.192932,207.887211,7751.950928
max,206.664978,217.206314,209.633942,271.525848,195.043945,254.974380,10.459358,273.582672,277.923584,11856.028320


# DF to CSV Pass 1

In [94]:
#lab_data.to_csv("Resources_clean/lab_pass1.csv")
#rx_data.to_csv("Resources_clean/rx_pass1.csv")
#ta_data.to_csv("Resources_clean/ta_pass1.csv")
#tb_data.to_csv("Resources_clean/tb_pass1.csv")
#tc_data.to_csv("Resources_clean/tc_pass1.csv")